# Setup / Imports

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../novelinsights/backend')

In [4]:
from novelinsights.schemas.knowledge.node import Node

# Split the ebook into proper chapters

In [15]:
from ebooklib import epub

file1 = "../data/frankenstein.epub"
file2 = "../data/grapes-of-wrath.epub"
file3 = "../data/outlaw-of-torn.epub"
book1 = epub.read_epub(file1, options={'ignore_ncx': True})
book2 = epub.read_epub(file2, options={'ignore_ncx': True})
book3 = epub.read_epub(file3, options={'ignore_ncx': True})

In [16]:
from ebook_tree_parser.toctree import TocTree

In [17]:
tree1 = TocTree(book1)
tree2 = TocTree(book2)
tree3 = TocTree(book3)

DEBUG:charset_normalizer:Encoding detection: ascii is most likely the one.
DEBUG:charset_normalizer:Encoding detection: ascii is most likely the one.
DEBUG:charset_normalizer:Encoding detection: utf_8 is most likely the one.
DEBUG:charset_normalizer:Encoding detection: utf_8 is most likely the one.
DEBUG:charset_normalizer:Encoding detection: utf_8 is most likely the one.
DEBUG:charset_normalizer:Encoding detection: utf_8 is most likely the one.
DEBUG:charset_normalizer:Encoding detection: utf_8 is most likely the one.
DEBUG:charset_normalizer:Encoding detection: utf_8 is most likely the one.
DEBUG:charset_normalizer:Encoding detection: utf_8 is most likely the one.
DEBUG:charset_normalizer:Encoding detection: utf_8 is most likely the one.
DEBUG:charset_normalizer:Encoding detection: utf_8 is most likely the one.
DEBUG:charset_normalizer:Encoding detection: utf_8 is most likely the one.
DEBUG:charset_normalizer:Encoding detection: utf_8 is most likely the one.
DEBUG:charset_normalizer:

In [18]:
# from novelinsights.wikigen_fiction.agents.read_chapter_agent import ReadChapterAgent

# read_chapter_agent = ReadChapterAgent(llm=None, db=None)
# print(read_chapter_agent.prompt(tree2[5].title, tree2[5].content))

# Use an LLM (through llamaIndex for consistency) to read the chapters and store in qdrant

In [44]:
import logging
logging.basicConfig(level=logging.DEBUG)

from novelinsights.utils import LLMWrapper

from novelinsights.wikigen_fiction.agents.read_chapter_agent import ReadChapterAgent, ReadChapterResponse, ReadChapterPayload
from novelinsights.wikigen_fiction.agents.core_summary_agent import CoreSummaryAgent, CoreSummaryResponse, CoreSummaryPayload
from llama_index.llms.anthropic import Anthropic
from llama_index.llms.openai import OpenAI
import tiktoken

# api key from .env file
import os
from dotenv import load_dotenv
load_dotenv(override=True)
# os.getenv("OPENAI_API_KEY")
# os.getenv("ANTHROPIC_API_KEY")

from novelinsights.db import QdrantDB
db = QdrantDB()

llm_wrapper = LLMWrapper(
    llm=Anthropic(
        api_key=os.getenv("ANTHROPIC_API_KEY"), 
        model="claude-3-5-sonnet-20240620",        
        # system_prompt="Let's think step by step.",
        max_tokens=3_200,
        ),
    mode="chat",
    tokenizer=Anthropic().tokenizer
    )

# llm_wrapper = LLMWrapper(
#     llm=OpenAI(
#         api_key=os.getenv("OPENAI_API_KEY"),
#         model="gpt-4o"
#         ),
#     mode="completion",
#     tokenizer=tiktoken.encoding_for_model("gpt-4o")
#     )

read_chapter_agent = ReadChapterAgent(llm=llm_wrapper, db=db)
core_summary_agent = CoreSummaryAgent(llm=llm_wrapper, db=db, book_title=book2.title)

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/Users/jimmyyao/miniconda3/envs/novel/lib/python3.11/site-packages/certifi/cacert.pem'
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /api/models/qdrant/bge-small-en-v1.5-onnx-q/revision/main HTTP/11" 307 92
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /api/models/Qdrant/bge-small-en-v1.5-onnx-Q/revision/main HTTP/11" 200 1444
Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 131072.00it/s]
DEBUG:httpcore.connection:connect_tcp.started host='localhost' port=6333 local_address=None timeout=5.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x14f8d9550>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'GET']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'GET']>
DEBUG

In [48]:
chap_num = 5
read_prompt = read_chapter_agent.get_prompt(chap_num, tree2[chap_num].title, tree2[chap_num].content)
print(read_prompt)

You are an AI assistant designed to analyze chapters of fiction.

# Task
Your task is to produce a comprehensive and detailed summary of the current chapter, as well as extract and organize all significant information into predefined categories. The output should be thorough and suitable for creating Wikipedia-style/fan wiki entries, ensuring that all essential details from the chapter are captured accurately.

# Current Chapter Context:
## Chapter Title: GRAPES OF WRATH CHAPTER I TOWARDS THE PUSH
## Chapter Content: 
---
##  GRAPES OF WRATH

## CHAPTER I  
TOWARDS THE PUSH

The rank and file of the 5/6 Service Battalion of the Stonewalls knew that “there was another push on,” and that they were moving up somewhere into the push; but beyond that and the usual crop of wild and loose-running rumors they knew nothing. Some of the men had it on the most exact and positive authority that they were for the front line and “first over the parapet”; others on equally positive grounds knew that 

In [49]:
# chap_num = 5
# read_prompt = read_chapter_agent.get_prompt(chap_num, tree2[chap_num].title, tree2[chap_num].content)
# print(read_prompt)

read_resp = read_chapter_agent.generate(chap_num, tree2[chap_num].title, tree2[chap_num].content)

# with open("../output/grapes-of-wrath/[5] GRAPES OF WRATH CHAPTER I TOWARDS THE PUSH/read_chapter_response.txt", "w") as f:
#     f.write(read_resp)

DEBUG:anthropic._base_client:Request options: {'method': 'post', 'url': '/v1/messages', 'files': None, 'json_data': {'max_tokens': 3200, 'messages': [{'role': 'user', 'content': [{'text': 'You are an AI assistant designed to analyze chapters of fiction.\n\n# Task\nYour task is to produce a comprehensive and detailed summary of the current chapter, as well as extract and organize all significant information into predefined categories. The output should be thorough and suitable for creating Wikipedia-style/fan wiki entries, ensuring that all essential details from the chapter are captured accurately.\n\n# Current Chapter Context:\n## Chapter Title: GRAPES OF WRATH CHAPTER I TOWARDS THE PUSH\n## Chapter Content: \n---\n##  GRAPES OF WRATH\n\n## CHAPTER I  \nTOWARDS THE PUSH\n\nThe rank and file of the 5/6 Service Battalion of the Stonewalls knew that “there was another push on,” and that they were moving up somewhere into the push; but beyond that and the usual crop of wild and loose-runn

IndexError: list index out of range

In [97]:
# chap_num = 5
# core_prompt = core_summary_agent.get_prompt(read_chapter_agent.response, None)
# print(core_prompt)

core_resp = core_summary_agent.generate(read_chapter_agent.response, None)

DEBUG:anthropic._base_client:Request options: {'method': 'post', 'url': '/v1/messages', 'files': None, 'json_data': {'max_tokens': 3200, 'messages': [{'role': 'user', 'content': [{'text': '# Task\nYour task is to read the provided summary of the chapter of the story and update the core summary according to the provided JSON schema. The updated summary should accurately reflect the story progression up to the current chapter, ensuring consistency and coherence with previous information, but changing or adding details as necessary.\n# Previous Core Summary\n(No previous core summary)\n\n# Chapter Summary\n---\n## Information Extraction\n\n### 2.1 Key Events or Developments\n\n1. The Stonewalls Battalion\'s Movement\n   - Name: The Stonewalls Battalion moves towards the front\n   - What happened: The 5/6 Service Battalion of the Stonewalls has been marching for three days towards the front lines. They are aware of an impending "push" but have limited concrete information about their role.

ValidationError: 2 validation errors for CoreSummaryPayload
static_metadata
  Field required [type=missing, input_value={'name': 'Grapes of wrath...'status': 'ongoing'}]})}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing
response
  Input should be a valid dictionary or instance of CoreSummaryResponse [type=model_type, input_value=CoreSummaryResponse(full_... 'status': 'ongoing'}]}), input_type=CoreSummaryResponse]
    For further information visit https://errors.pydantic.dev/2.9/v/model_type

In [104]:
print(core_summary_agent.agent_history[0].prompt)

# Task
Your task is to read the provided summary of the chapter of the story and update the core summary according to the provided JSON schema. The updated summary should accurately reflect the story progression up to the current chapter, ensuring consistency and coherence with previous information, but changing or adding details as necessary.
# Previous Core Summary
(No previous core summary)

# Chapter Summary
---
## Information Extraction

### 2.1 Key Events or Developments

1. The Stonewalls Battalion's Movement
   - Name: The Stonewalls Battalion moves towards the front
   - What happened: The 5/6 Service Battalion of the Stonewalls has been marching for three days towards the front lines. They are aware of an impending "push" but have limited concrete information about their role. The battalion halts in a small orchard for a night's bivouac.
   - Who was involved: The entire Stonewalls Battalion
   - Where: Near the front lines, initially in a small orchard
   - When: Over the co

In [7]:


with open("../output/grapes-of-wrath/[5] GRAPES OF WRATH CHAPTER I TOWARDS THE PUSH/read_chapter_response.txt") as f:
    read_chapter_agent._mock_generate(f.read(), 5, tree2[5].title, tree2[5].content, None)
    
with open("../output/grapes-of-wrath/[5] GRAPES OF WRATH CHAPTER I TOWARDS THE PUSH/core_summary_response.txt") as f:
    core_summary_agent._mock_generate(f.read(), read_chapter_agent.response, None)

DEBUG:root:prompt tokens: 4085
DEBUG:root:response tokens: 1675
DEBUG:root:prompt tokens: 2355
DEBUG:root:response tokens: 0


In [11]:
core_summary_agent.payload

CoreSummaryPayload(type='core_summary', name='Grapes of wrath', prompt='# Task\nYour task is to read the provided summary of the chapter of the story and update the core summary according to the provided JSON schema. The updated summary should accurately reflect the story progression up to the current chapter, ensuring consistency and coherence with previous information, but changing or adding details as necessary.\n# Previous Core Summary\n(No previous core summary)\n\n# Chapter Summary\n---\n## Information Extraction\n\n### Entities\n\n1. Larry Arundel\n   - Type: Character\n   - Role: Soldier in the 5/6 Service Battalion of the Stonewalls\n   - Background: From a good family, previously worked in his father\'s office in London\n   - Significance: High\n   - Reason: Provides insight into the military operations and explains artillery tactics to his comrades\n\n2. Billy Simson\n   - Type: Character\n   - Role: Soldier in the 5/6 Service Battalion of the Stonewalls\n   - Background: Ed

In [8]:
llm_wrapper.estimate_tokens(f"{read_chapter_agent.payload.model_dump()['response']['info_extraction']}")

1934

In [9]:
read_chapter_agent.payload.model_dump()['response']['info_extraction']

{'chapter_summary': '## Chapter Summary\n\nThe 5/6 Service Battalion of the Stonewalls is moving towards the front line in preparation for a major offensive, known as "the push." As they march, the sound of artillery fire grows increasingly intense. After three days of marching, the battalion halts in a small orchard, intending to set up a night\'s bivouac. However, their position comes under fire from German artillery, forcing them to relocate to nearby farm buildings for better shelter.\n\nThe narrative focuses on four soldiers from diverse backgrounds who have formed an unlikely friendship: Larry Arundel, Billy Simson, Ben "Pug" Sneath, and Jefferson "Kentucky" Lee. These men represent different social classes and nationalities, highlighting the mixing of backgrounds in the wartime army.\n\nAfter relocating to the farm buildings, the soldiers observe a nearby allied artillery battery in action. The battery is engaged in "wire cutting," using their guns to destroy German barbed wire 

In [11]:
core_summary_agent.payload

In [21]:
core_summary_agent.last_response.core_summary_json

AttributeError: 'CoreSummaryAgent' object has no attribute 'last_response'

## Qdrant


In [176]:
from novelinsights.db import QdrantDB

db = QdrantDB()

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 72315.59it/s]


In [171]:
# docs = ["Random text"]
# metadata = [
#     {"chapter": 5, "title": "GRAPES OF WRATH CHAPTER I TOWARDS THE PUSH"}
# ]
# client.add(
#     collection_name="test_collection",
#     documents=docs,
#     metadata=metadata,
#     ids=[0]
# )
# 
# https://qdrant.tech/documentation/concepts/payload/#update-payload
# client.set_payload(
#     collection_name="test_collection",
#     payload={"title": "TOWARDS THE PUSH", "newfield": "newvalue"},
#     points=[0]
# )

UpdateResult(operation_id=7, status=<UpdateStatus.COMPLETED: 'completed'>)

# Prepare folders to store outputs

In [127]:
# make directory for the output, if it exists, create a new one with incremented number
import os

output_dir = "../output/frankenstein"
if os.path.exists(output_dir):
    i = 1
    while os.path.exists(output_dir + "_" + str(i)):
        i += 1
    output_dir += "_" + str(i)
os.makedirs(output_dir)

# make folders "inprompt", "json", and "response" folders in the output directory
os.makedirs(os.path.join(output_dir, "inprompt"))
os.makedirs(os.path.join(output_dir, "json"))
os.makedirs(os.path.join(output_dir, "response"))

In [57]:
# message = client.messages.create(
#     model="claude-3-5-sonnet-20240620",
#     max_tokens=1000,
#     messages=[
#         {
#             "role": "user",
#             "content": [
#                 {
#                     "type": "text",
#                     "text": template(tree[0].content)
#                 }
#             ]
#         }
#     ]
# )
# print(message.content[0].text)

Here's the JSON created based on the provided excerpt and schema:

```json
{
  "novelInfo": {
    "title": "Frankenstein; Or, The Modern Prometheus",
    "author": "Mary Wollstonecraft Shelley",
    "genre": ["Gothic", "Science Fiction", "Horror"]
  },
  "summary": "Frankenstein; Or, The Modern Prometheus is a novel that explores themes of scientific ambition, the consequences of creating life, and the responsibilities of a creator to their creation. The story likely follows the journey of a scientist who creates a living being, leading to unforeseen and potentially disastrous consequences.",
  "characters": [
    {
      "name": "Victor Frankenstein",
      "description": "Presumed to be the main character, a scientist who likely creates artificial life.",
      "role": "protagonist"
    },
    {
      "name": "The Creature",
      "description": "Presumed to be the artificial being created by Victor Frankenstein.",
      "role": "antagonist"
    }
  ],
  "plotPoints": [],
  "worldBui

In [46]:
# completion = client.chat.completions.create(
#     model="chatgpt-4o-latest",
#     messages=[
#         {"role": "user", "content": template(tree[0].content)}
#     ]
# )
# print(completion.choices[0].message.content)

In [129]:
from novelinsights.prompts.wikigen.fiction import template

prev_json = None
for i, excerpt in enumerate(tree):
    print(f"[{i}]\t{excerpt.title}")
    
    cur_template = template(excerpt, prev_json=prev_json)
    
    with open(os.path.join(output_dir, "inprompt", f"[{i}] {excerpt.title}.txt"), "w") as f:
        f.write(cur_template)
    
        message = client.messages.create(
            model="claude-3-5-sonnet-20240620",
            max_tokens=8192,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": cur_template
                        }
                    ]
                }
            ]
        )
        
        with open(os.path.join(output_dir, "response", f"[{i}] {excerpt.title}.txt"), "w") as f:
            f.write(message.content[0].text)
        
        try:
            prev_json = parse_json(message.content[0].text)
        except:
            pass
        
        with open(os.path.join(output_dir, "json", f"[{i}] {excerpt.title}.json"), "w") as f:
            f.write(json.dumps(prev_json, indent=4))
            

[0]	Frankenstein;
[1]	or, the Modern Prometheus
[2]	CONTENTS
[3]	Letter 1
[4]	Letter 2
[5]	Letter 3
[6]	Letter 4
[7]	Chapter 1
[8]	Chapter 2
[9]	Chapter 3
[10]	Chapter 4
[11]	Chapter 5
[12]	Chapter 6


KeyboardInterrupt: 